In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import osmnx as ox

In [ ]:
# area = ox.geocode_to_gdf("Warszawa, Polska")
area = ox.geocode_to_gdf("Wrocław, Poland")

In [ ]:
gdf_wgs84 = area.to_crs(epsg=4326)
gdf_wgs84.plot()

In [ ]:
from srai.loaders.osm_tag_loader.osm_tag_loader import OSMTagLoader
from srai.loaders.osm_tag_loader.filters.hex2vec import HEX2VEC_FILTER

h2v_tags = HEX2VEC_FILTER

In [ ]:
loader = OSMTagLoader()
results = loader.load(gdf_wgs84, h2v_tags)
results

In [ ]:
from itertools import product
from functional import seq

seq(h2v_tags.items()).starmap(
    lambda k, v: product([k], [v] if isinstance(v, bool) else v)
).flatten().list()

In [ ]:
loader = OSMTagLoader()
results = loader.load(gdf_wgs84, h2v_tags)

In [ ]:
results

In [ ]:
results.query("amenity.notna()").plot()

In [ ]:
gdf_wgs84.plot()

In [ ]:
results

In [ ]:
gdf = ox.geometries_from_polygon(gdf_wgs84.loc[0, "geometry"], tags={"leisure": "park"})
gdf

In [ ]:
results

In [ ]:
gdf[["geometry", "leisure"]]

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame()

In [ ]:
from srai.loaders.osm_tag_loader.filters.popular import get_popular_tags

popular_tags = get_popular_tags(True, min_fraction=0.0005)
popular_tags

In [ ]:
from srai.loaders.osm_tag_loader.filters.hex2vec import HEX2VEC_FILTER

h2v_tags = HEX2VEC_FILTER

In [ ]:
ox.settings.timeout = 10000

In [ ]:
import pandas as pd

In [ ]:
h2v_tags

In [ ]:
from json import JSONDecodeError
import json
import sys

geometries = []
for polygon in gdf_wgs84["geometry"]:
    for key, values in h2v_tags.items():
        print(key)
        tags = {key: values}
        tag_geoms = ox.geometries_from_polygon(polygon, tags)
        geometries.append(tag_geoms)

geometries = pd.concat(geometries)
geometries

In [ ]:
grouped = geometries.groupby(["element_type", "osmid"])

In [ ]:
i = 0
for group_id, group_df in grouped:
    if len(group_df) > 1:
        i += 1
        if i == 3:
            break
group_df

In [ ]:
group_df.iloc[0][group_df.iloc[0].notna()].index

In [ ]:
group_df.iloc[1][group_df.iloc[1].notna()].index

In [ ]:
len(group_df)

In [ ]:
geometries_squashed = geometries.groupby(["element_type", "osmid"]).first()

In [ ]:
geometries_squashed

In [ ]:
geometries_all = []
for polygon in gdf_wgs84["geometry"]:
    tag_geoms = ox.geometries_from_polygon(polygon, popular_tags)
    geometries_all.append(tag_geoms)
geometries_all = pd.concat(geometries_all)
geometries_all

In [ ]:
geometries_all.sort_index().sort_index(axis=1) == geometries_squashed.sort_index().sort_index(
    axis=1
)

In [ ]:
geometries.groupby(["element_type", "osmid"]).first().query("amenity.notna() and leisure.notna()")

In [ ]:
geometries[0].reset_index()["osmid"].nunique()

In [ ]:
filtered_geoms = []
for df, key in zip(geometries, popular_tags):
    if df.empty:
        continue
    filtered_geoms.append(df[["geometry", key]])
filtered_geoms

In [ ]:
[k for k in popular_tags if k in filtered_geoms.columns]

In [ ]:
features_gdf = results.reset_index(level=0, drop=True)
features_gdf.index.rename("feature_id", inplace=True)
features_gdf

In [ ]:
features_gdf = features_gdf.set_crs(epsg=4326)

In [ ]:
from srai.regionizers import H3Regionizer

regionizer = H3Regionizer(resolution=8)
regions_gdf = regionizer.transform(gdf_wgs84)
regions_gdf.index.name = "region_id"
regions_gdf

In [ ]:
regions_gdf.plot()

In [ ]:
from srai.joiners import IntersectionJoiner

joiner = IntersectionJoiner()
joint_gdf = joiner.transform(regions_gdf, features_gdf)

In [ ]:
joint_gdf

In [ ]:
xd = [f"{k}_{v}" for k, vs in HEX2VEC_FILTER.items() for v in vs]

In [ ]:
from srai.embedders.count_embedder import CountEmbedder

embedder = CountEmbedder(expected_output_features=xd)
embeddings = embedder.transform(regions_gdf, features_gdf, joint_gdf)
embeddings

In [ ]:
from tqdm import tqdm

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]